## Working with multiple objects in the Python SDK
AIS supports multi-object operations on groups of objects. An `ObjectGroup` can be created with one of:
- a list of object names
- an [ObjectRange](https://github.com/NVIDIA/aistore/blob/master/python/aistore/sdk/object_range.py)
- a string template.

### Setup

In [ ]:
pip install aistore

In [ ]:
# Set up the client and create necessary buckets
from aistore import Client
from aistore.sdk.errors import AISError
from pathlib import Path
import hashlib
import os

ais_url = os.getenv("AIS_ENDPOINT", "http://localhost:8080")
client = Client(ais_url)
bucket = client.bucket("my-bck")
copy_dest_bucket = client.bucket("copy-destination-bucket")
transform_dest_bucket = client.bucket("transform-destination-bucket")
for bck in [bucket, copy_dest_bucket, transform_dest_bucket]:
    try:
        bck.create()
    except AISError as err:
        print(err)

In [ ]:
# Create some objects in the bucket
object_names = [f"example_obj_{i}" for i in range(10)]
for name in object_names:
    bucket.object(name).put_content("object content".encode("utf-8"))

### Create Object Group by list of names

In [ ]:
my_objects = bucket.objects(obj_names=object_names)

### Create Object Group by ObjectRange

In [ ]:
from aistore.sdk.object_range import ObjectRange

my_object_range = ObjectRange(prefix="example_obj_", min_index=1, max_index=3)
my_objects = bucket.objects(obj_range=my_object_range)

### Create Object Group by Template String
String templates can be passed directly to AIS following the [syntax described here](https://github.com/NVIDIA/aistore/blob/master/docs/batch.md#operations-on-multiple-selected-objects)

In [ ]:
# Equivalent to the range above
my_object_template = "example_obj_{1..3}"
my_objects = bucket.objects(obj_template=my_object_template)
# More advanced template example with multiple ranges and defined steps
complex_range = "example_obj_{0..10..2}_details_{1..9..2}.file-extension"

### Prefetch, Evict, and Delete with multiple Objects

In [ ]:
# Prefetch and Evict only work for cloud backends
my_objects.prefetch()
my_objects.evict()

In [ ]:
before_deletion = bucket.list_all_objects()
delete_job_id = my_objects.delete()
client.job(delete_job_id).wait()
after_deletion = bucket.list_all_objects()
print(
    f"Objects before deletion: {len(before_deletion)}, objects after deletion: {len(after_deletion)}"
)

### Copy multiple objects

Copies selected objects directly to the new bucket

In [ ]:
copy_job = my_objects.copy(to_bck=copy_dest_bucket.name)
# The job will reach an idle state before finishing, so wait for idle
client.job(job_id=copy_job).wait_for_idle()
# See the objects in the destination bucket
copy_dest_bucket.list_all_objects()

#### Transform -- Provide an ETL to be performed on each object so the result appears in the destination bucket.

In [ ]:
# First create an ETL
# This is a simple example transform that reverses each object's contents (assuming utf-8 encoded text)
def transform(input_bytes):
    reversed_in_str = input_bytes.decode("utf-8")[::-1]
    return reversed_in_str.encode()


etl_name = "multiobj-transform-example"
try:
    client.etl().init_code(transform=transform, etl_name=etl_name)
except AISError as err:
    print(err)

# Now run the transform with the etl name specified
transform_job = my_objects.transform(
    etl_name=etl_name, to_bck=transform_dest_bucket.name
)
client.job(job_id=transform_job).wait_for_idle()

# The output will be in the destination bucket
transformed_objs = transform_dest_bucket.list_all_objects()

# See the result
for entry in transformed_objs:
    input_data = bucket.object(entry.name).get().read_all()
    output_data = transform_dest_bucket.object(entry.name).get().read_all()
    print(f"Object {entry.name} {input_data} => {output_data}")

In [ ]:
# Cleanup buckets
for bck in [bucket, copy_dest_bucket, transform_dest_bucket]:
    try:
        bck.delete()
    except AISError as err:
        print(err)